In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import json
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
from params import *
from utils.metrics import boxes_f1_score, precision_calc, get_boxes_from_df

## Load

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")
df_train['truth'] = (df_train['impact'] == 1) & (df_train['confidence'] > 1) & (df_train['visibility'] > 0) 


df_val = df_train[df_train["val_idx"] == 0]

In [ ]:
videos = df_val['video'].unique()

In [ ]:
PREDS_PATH = OUT_DIR + '22_12/'
epoch = 9
preds = pd.read_csv(PREDS_PATH + f"pred_0_fold_epoch_{epoch}_score_001_aug.csv")

In [ ]:
preds = preds[preds['pred'] > 0.1].reset_index(drop=True)

In [ ]:
df_pred = preds.copy()
# df_pred = pd.read_csv('../output/df_preds.csv')

# Classifier 3D inference

In [ ]:
from inference.classifier_3d import *

### Data

In [ ]:
if "nb_frame" not in df_pred.columns:
    df_max_frame = df_pred[['video', 'frame']].groupby('video').max().rename(columns={"frame": "nb_frame"}).reset_index()
    df_pred = df_pred.merge(df_max_frame, on="video")

In [ ]:
df_pred_vid = df_pred[df_pred["video"] == df_pred['video'].unique()[1]]

dataset = NFLDatasetClsInference3D(
    df_pred_vid,
    visualize=True,
    stride=2,
    n_frames=9,
    root=IMG_PATH_F,
)

### Main

In [ ]:
images = df_pred["image_name"].unique()

In [ ]:
# Where to load weights from
CP_FOLDER = "../logs_cls_3d/2021-01-02/13/"

# Models to use
configs = { 
    "slowonly": {
        "name": "slowonly",
        "num_classes": 1,
        "num_classes_aux": 0,
        "k": 5,
        "stride": 2,
        "num_frames": 9
    },
}

In [ ]:
models = []


for model in configs:
    models += [retrieve_model(configs[model], fold=0, log_folder=CP_FOLDER)]
    
    
preds = []
for vid in tqdm(df_pred['video'].unique()):
    df_pred_vid = df_pred[df_pred["video"] == vid]

    pred = inference(df_pred_vid, models, root=IMG_PATH_F, stride=2, n_frames=9, batch_size=128)
    preds.append(pred)
    
df_pred['pred_cls_3d'] = np.concatenate(preds)

In [ ]:
df_pred.to_csv('../output/df_preds.csv', index=False)

# Post-processing

In [ ]:
from post_processing.adjacency import post_process_adjacency
from post_processing.expansion import expand_boxes
from post_processing.view import post_process_view

In [ ]:
df_pred = pd.read_csv('../output/df_preds.csv')

In [ ]:
df_pred['scores'] = df_pred['pred']

In [ ]:
# Blend weights 

weights = {
    'pred_cls_3d_29_6': 0.25,   # r18 ext
    'pred_cls_3d_30_1': 0.25,   # r34
    'pred_cls_3d_30_0': 0.25,   # r18
    'pred_cls_3d_30_3': 0.25,   # r18 aux
    'pred_cls_3d_02_5': 0.66,   # i3d
    'pred_cls_3d_02_7': 0.66,   # slowfast
    'pred_cls_3d_02_13': 0.66,  # slowonly
}

In [ ]:
df_pred['pred_cls_blend'] = 0

for col in weights:
    df_pred['pred_cls_blend'] += df_pred[col] * weights[col]
    
df_pred['pred_cls_blend'] /= np.sum(list(weights.values()))

### Params

In [ ]:
DET_THRESHOLD = 0.35
CLS_THRESHOLD = 0.48

# Change threshold after frame
SWITCH_FRAME = 150
DET_THRESHOLD2 = 0.40
CLS_THRESHOLD2 = 0.65

# Lower thresholds for sideline
DELTA_CLS = -0.07
DELTA_DET = -0.05

# Adjacency post-processing
NMS_THRESHOLD = 0.41
MAX_FRAME_DIST = 9
N_TIMES = 1

# View post-processing
MIN_DIST = 4
VIEW_THRESHOLD = 0.86  

# Boxes expansion
R = 0.22

### Thresholding

In [ ]:
df_pred_pp = df_pred.copy()

df_pred_pp1 = df_pred_pp.loc[
    (df_pred_pp.scores > DET_THRESHOLD)
    & (df_pred_pp.frame <= SWITCH_FRAME)
    & (df_pred_pp.view == "Endzone")
]
df_pred_pp2 = df_pred_pp.loc[
    (df_pred_pp.scores > DET_THRESHOLD2)
    & (df_pred_pp.frame > SWITCH_FRAME)
    & (df_pred_pp.view == "Endzone")
]
df_pred_pp3 = df_pred_pp.loc[
    (df_pred_pp.scores > DET_THRESHOLD - DELTA_DET)
    & (df_pred_pp.frame <= SWITCH_FRAME)
    & (df_pred_pp.view == "Sideline")
]
df_pred_pp4 = df_pred_pp.loc[
    (df_pred_pp.scores > DET_THRESHOLD2 - DELTA_DET)
    & (df_pred_pp.frame > SWITCH_FRAME)
    & (df_pred_pp.view == "Sideline")
]
df_pred_pp = pd.concat([df_pred_pp1, df_pred_pp2, df_pred_pp3, df_pred_pp4], axis=0).reset_index(drop=True)

In [ ]:
df_pred_pp_cls1 = df_pred_pp.copy()[
    (df_pred_pp["pred_cls_blend"] > CLS_THRESHOLD)
    & (df_pred_pp.frame <= SWITCH_FRAME)
    & (df_pred_pp.view == "Endzone")
]
df_pred_pp_cls2 = df_pred_pp.copy()[
    (df_pred_pp["pred_cls_blend"] > CLS_THRESHOLD2)
    & (df_pred_pp.frame > SWITCH_FRAME)
    & (df_pred_pp.view == "Endzone")
]
df_pred_pp_cls3 = df_pred_pp.copy()[
    (df_pred_pp["pred_cls_blend"] > CLS_THRESHOLD - DELTA_CLS)
    & (df_pred_pp.frame <= SWITCH_FRAME)
    & (df_pred_pp.view == "Sideline")
]
df_pred_pp_cls4 = df_pred_pp.copy()[
    (df_pred_pp["pred_cls_blend"] > CLS_THRESHOLD2 - DELTA_CLS)
    & (df_pred_pp.frame > SWITCH_FRAME)
    & (df_pred_pp.view == "Sideline")
]
df_pred_pp = pd.concat(
    [df_pred_pp_cls1, df_pred_pp_cls2, df_pred_pp_cls3, df_pred_pp_cls4], axis=0
).reset_index(drop=True)

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

### Box expansion

In [ ]:
df_pred_pp = expand_boxes(df_pred_pp, r=R)

### Adjacency Post-processing

In [ ]:
for i in range(N_TIMES):
    df_pred_pp = post_process_adjacency(
        df_pred_pp,
        threshold=THRESHOLD_IOU,
        max_dist=MAX_DIST,
        min_clust_size=MIN_CLUST_SIZE,
    )

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')

### View PP

In [ ]:
if MIN_DIST > 0:
    df_pred_pp_view = post_process_view(
        df_pred_pp, 
        min_dist=MIN_DIST,
        threshold=VIEW_THRESHOLD,
        cls_col="pred_cls_blend",
    )

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp_view, videos)
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> CV score is {score:.4f}')